In [1]:
# Define embeddings model - Voyage AI
import os
import voyageai

API_KEY_FILEPATH = r"C:\Users\rotem\OneDrive - Afeka College Of Engineering\Final Project\Portions Estimation\Voyage AI API Key.txt"
with open(API_KEY_FILEPATH, "r") as f:
    API_KEY = f.read()

os.environ["VOYAGE_API_KEY"] = API_KEY
vo = voyageai.Client()

C:\Users\rotem\PycharmProjects\GlycemicLoad\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text = "['Brussels sprouts', 'Celery', 'Black olives']"
result = vo.embed(text, model="voyage-3-lite", input_type="query")
print(result.embeddings[0])

[-0.01698211580514908, -0.007467241026461124, 0.03430747240781784, 0.0078115821816027164, 0.05023152008652687, -0.10064349323511124, -0.06125707924365997, -0.013829641975462437, -0.04165331646800041, -0.09522242099046707, -0.022479958832263947, -0.025914743542671204, 0.03713438659906387, 0.10511588305234909, 0.027300579473376274, 0.06534314155578613, -0.09716398268938065, -0.062130268663167953, 0.007745659910142422, 0.020806701853871346, 0.07511065900325775, 0.03880932182073593, -0.020555533468723297, 0.034650709480047226, -0.01040463987737894, 0.04787784069776535, -0.030549542978405952, 0.026385772973299026, 0.04610052704811096, 0.015483954921364784, -0.00209624832496047, -0.03761167451739311, 0.021138982847332954, 0.04872709885239601, 0.021214939653873444, 0.03536369651556015, 0.0014679632149636745, 0.0626605972647667, -0.06656141579151154, 0.05173299461603165, -0.04177692160010338, -0.084227055311203, 0.020120596513152122, -0.037490881979465485, 0.038430362939834595, 0.0275856275111

HuggingFace - MiniLM-L6-V2

In [9]:
import pandas as pd
import os
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [6]:
# Data handling
CURRENT_DIR = os.path.dirname(os.getcwd())
CLAUDE_INGR_DATA_PATH = os.path.join(CURRENT_DIR, r"data\raw\ClaudeResults.csv")
claude_results_df = pd.read_csv(CLAUDE_INGR_DATA_PATH)
claude_ingr_data = claude_results_df['Predicted Ingredients'].values.tolist()
claude_ingr_data

['Olives',
 'Brussels sprouts, Celery, Black olives',
 'Brussels sprouts, Kalamata olives, Celery',
 'Brussels sprouts, Olives, Celery',
 'Brussels sprouts, Olives, Celery',
 'Coleslaw',
 'Sweet Potato',
 'Sweet Potatoes, Scrambled Eggs, Strawberries',
 'Lettuce, Eggplant, Garlic',
 'Rice, Coconut',
 'Steak pieces - appears to be cooked medium-rare beef steak cut into chunks on a white plate.',
 'Avocado slices',
 'Lettuce, Cucumber, Black olives',
 'Raspberries',
 'Scrambled eggs, Bacon, Broccoli, Strawberries, Cherry tomatoes',
 'Sweet potato chunks',
 'Sweet potato, Scrambled eggs, Raspberries, Strawberry, Blackberries',
 'Honeydew melon',
 'Cantaloupe',
 'Bacon, Cantaloupe, Honeydew melon',
 'Watermelon',
 'Bacon, Sweet Potato, Watermelon',
 'Watermelon cubes, Roasted potatoes, Bacon strips',
 'Cantaloupe, Watermelon, Honeydew melon',
 'Roasted Potatoes',
 'Potatoes, Cucumber, Bacon',
 'Bacon, Sweet Potato, Cantaloupe',
 'Roasted Potatoes, Honeydew Melon, Bacon',
 'Bacon',
 'Waterm

In [7]:
# Define model and perform embeddings
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(claude_ingr_data)
embeddings

C:\Users\rotem\PycharmProjects\GlycemicLoad\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rotem\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


array([[-0.04500768,  0.0034053 , -0.02296103, ...,  0.02707663,
         0.08980288,  0.05819259],
       [-0.02350855, -0.07336354, -0.05097178, ...,  0.07236314,
         0.0571612 ,  0.04664182],
       [-0.00914181, -0.06210283, -0.04262816, ...,  0.07676855,
         0.07304444,  0.05296791],
       ...,
       [-0.0641641 ,  0.03593367, -0.00939612, ...,  0.00926099,
         0.03549637, -0.03510123],
       [-0.02171859, -0.01594793, -0.01133736, ..., -0.0271293 ,
         0.06965268, -0.01535775],
       [-0.01937638, -0.03595918, -0.00377372, ..., -0.00717697,
         0.03240446,  0.08734068]], dtype=float32)

In [1]:
# t-SNE
def visualize_embeddings(embeddings, labels):
    tsne = TSNE(n_components=2, random_state=42)
    reduced_embeddings = tsne.fit_transform(embeddings)
    plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c='blue')
    for i, label in enumerate(labels):
        plt.annotate(label, (reduced_embeddings[i, 0], reduced_embeddings[i, 1]))
    plt.show()
